Ce notebook represente une version obselete du projet. Voir le notebook objet.ipynb

In [1]:
import pandas as pd
import numpy as np
from itertools import product, combinations
import copy
from tqdm import tqdm
from tools import *

In [2]:
df= pd.DataFrame({
    "N_candidat": ["0001", "0002", "0003", "0004", "0005"],
    "S1":[15, 11, 14, 16, 11],
    "S2": [13, 5, 12, 15, 11],
    "S3":[16, 8, 11, 15, 11],
    "S4": [17, 12, 15, 18, 11],
    "S5": [13, 10, 13, 15, 11],

    "Logique": [1, 1, 1, 0, 0],
    "BDD": [1, 1, 1, 0, 0],
    "PS": [1, 1, 1, 1, 0],
    "Moyenne": [2, 0, 1, 2, 0],
    "Eligible": [True, True, True, False, False],
    "Decision": ['A','R','L','R','R']
})
V = list(df.columns)[1:-1]
display(V, df)

['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS', 'Moyenne', 'Eligible']

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


# ACi
## AC1

In [3]:
def AC1(N, X, decision, df=df):
    c = df[df.N_candidat == N].copy()
    #ligne existe, la décision est bonne et toutes les variables de X ont les bonnes valeurs aussi
    return len(c) == 1 and  c.Decision.iloc[0] == decision and all((str(c.loc[:, [a]].iloc[0, 0]) == str(X[a])) for a in X)


In [4]:
def test_AC1():
    display(df)
    # Cas de succès
    assert AC1("0001", {"Logique": 1}, 'A')
    assert AC1("0001", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A')
    assert AC1("0004", {"S2":15, "BDD":0}, 'R')

    # Cas d'échec
    assert not AC1("xxxx", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A')
    display("AC1 est bien correcte.")

test_AC1()

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


'AC1 est bien correcte.'

## AC2

In [5]:
def AC2(N_candidat, X, decision, df=df):
    candidat=df[df.N_candidat==N_candidat].copy()
    if (len(candidat)==0):
        return False
    vals=candidat.iloc[0].to_dict()
    X_var=list(X.keys())

    variable_domains = {
        "S1": range(0, 21),
        "S2": range(0, 21),
        "S3": range(0, 21),
        "S4": range(0, 21),
        "S5": range(0, 21),
        "BDD": [0, 1],
        "Logique": [0, 1],
        "PS": [0, 1],
        "Moyenne":[0,1,2],
        "Eligible":[True,False]
    }
    def generate_combinations(X, variable_domains):
        # Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
        combinations = product(*[variable_domains[var] for var in X])

        # Exclure la combinaison identique à X
        return [combination for combination in combinations if combination != tuple(X.values())]

    combinations = generate_combinations(X, variable_domains)

    # Vérifier chaque combinaison
    for combination in combinations:
        x_prime = dict(zip(X_var, combination))
        for k in x_prime.keys():
            vals[k]=x_prime[k]
        m,e,d=med(**{k: v for k, v in vals.items() if k in ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']})

        if ('Eligible' in x_prime and x_prime['Eligible']!=e) or ('Moyenne' in x_prime and x_prime['Moyenne']!=m) :
            continue
        if d!=decision:
            #print(f"AC2 est vérifié pour une affectation x_prime: {x_prime}")
            return True
    return False

In [6]:
def test_AC2():
	display(df)
	# Cas de succès
	assert AC2("0001", {"S1": 15},'A')
	assert AC2("0002", {"S1": 13, "S5": 11}, 'R')
	assert AC2("0003", {"Logique": 1, "S5": 11, "S2":8},'L')

	# Cas d'échec
	assert not AC2("0004", {"S5": 11, "BDD": 0},'R')
	assert not AC2("0005", {"S1": 13, "BDD": 0}, 'R')
	print("AC2 est bien correcte")

test_AC2()



,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


AC2 est bien correcte


Version améliorée: ca ameliore les cas où AC3 n'est pas vérifié tout en ayant une meme complexité dans le cas où AC2 et AC3 sont verifiés

In [27]:
def AC2_beta(N_candidat, X, decision, df=df):
    candidat=df[df.N_candidat==N_candidat].copy()
    if (len(candidat)==0):
        return False
    variable_domains = {
        "S1": range(0, 21),
        "S2": range(0, 21),
        "S3": range(0, 21),
        "S4": range(0, 21),
        "S5": range(0, 21),
        "BDD": [0, 1],
        "Logique": [0, 1],
        "PS": [0, 1],
        "Moyenne":[0,1,2],
        "Eligible":[True,False]
    }

    def subdict_combinations(dictionary):
        if not dictionary:
            return [{}]

        key = next(iter(dictionary))
        rest = dictionary.copy()
        del rest[key]

        subcombos = subdict_combinations(rest)
        result = []
        for subcombo in subcombos:
            result.append(subcombo)
            with_key = {key: dictionary[key]}
            result.append({**with_key, **subcombo})
        return result

    def generate_combinations(X, variable_domains):
        # Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
        combinations = product(*[variable_domains[var] for var in X])

        # Exclure la combinaison identique à X
        return [combination for combination in combinations if combination != tuple(X.values())]
    sub=subdict_combinations(X)
    for subX in sub:
        if len(subX):
            X_var=list(subX.keys())
            vals=candidat.iloc[0].to_dict()
            combinations = generate_combinations(subX, variable_domains)
            # Vérifier chaque combinaison
            for combination in combinations:
                x_prime = dict(zip(X_var, combination))
                for k in x_prime.keys():
                    vals[k]=x_prime[k]
                m,e,d=med(**{k: v for k, v in vals.items() if k in ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']})

                if ('Eligible' in x_prime and x_prime['Eligible']!=e) or ('Moyenne' in x_prime and x_prime['Moyenne']!=m) :
                    continue
                if d!=decision and len(x_prime)==len(X):
                    print("AC2 et AC3 est vérifié pour une affectation x_prime:",x_prime, " la decision est :", d)
                    return True
                elif d!=decision:
                    print("Erreur : AC2 et AC3 est vérifié pour une affectation x_prime :", x_prime, " la decision est :", d)
                    return False
    print("Erreur : AC2 n'est pas vérifiée, veuillez saisir un superset de l'ensemble des variables sélectionné")
    return False


In [28]:
def test_AC2_beta():
	# Cas de succès
	assert AC2_beta("0001", {"S1": 15},'A')
	assert AC2_beta("0002", {"S1": 11, "S5": 10}, 'R')
	assert not AC2_beta("0003", {"Logique": 1, "S5": 13, "S2":12},'L')

	# Cas d'échec
	assert not AC2_beta("0004", {"S5": 15, "BDD": 0},'R')
	assert not AC2_beta("0005", {"S1": 11, "BDD": 0}, 'R')
	print("AC2 est bien correcte")

test_AC2_beta()



AC2 et AC3 est vérifié pour une affectation x_prime: {'S1': 0}  la decision est : R
AC2 et AC3 est vérifié pour une affectation x_prime: {'S1': 15, 'S5': 20}  la decision est : L
Erreur : AC2 et AC3 est vérifié pour une affectation x_prime : {'Logique': 0}  la decision est : R
Erreur : AC2 n'est pas vérifiée, veuillez saisir un superset de l'ensemble des variables sélectionné
Erreur : AC2 n'est pas vérifiée, veuillez saisir un superset de l'ensemble des variables sélectionné
AC2 est bien correcte


In [29]:
def test_AC2():
	# Cas de succès
	assert AC2("0001", {"S1": 15},'A')
	assert AC2("0002", {"S1": 13, "S5": 11}, 'R')
	assert AC2("0003", {"Logique": 1, "S5": 11, "S2":8},'L')

	# Cas d'échec
	assert not AC2("0004", {"S5": 11, "BDD": 0},'R')
	assert not AC2("0005", {"S1": 13, "BDD": 0}, 'R')
	print("AC2 est bien correcte")

test_AC2()



AC2 est bien correcte


## AC3:

In [30]:


def AC3(N, X, decision ,df=df):
    keys = list(X.keys())
    subsets = [dict(zip(combination, (X[key] for key in combination))) for r in range(1, len(keys)) for combination in combinations(keys, r)]
    return all(not AC1(N, subset, decision, df) or not AC2(N, subset, decision) for subset in subsets)


In [31]:
def test_AC3():
	display(df)
	# Cas de succès
	assert AC3("0001", {"Logique": 1}, 'A')
	assert AC3("0004", {"Logique": 0, "BDD": 0}, 'R')
	assert AC3("0004", {"Logique": 0, "S1": 16}, 'R')

	# Cas d'échec
	assert not AC3("0001", {"Logique": 1, "PS": 1}, 'A')
	assert not AC3("0004", {"Logique": 0,  "BDD": 0,"S1": 16}, 'R')
	print("AC3 est bien correcte")

test_AC3()


,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


AC3 est bien correcte


In [32]:
def AC(N, X, decision, df=df, afficher = False):
  res1=AC1(N, X, decision, df)
  res2=AC2(N, X, decision, df)
  res3=AC3(N, X, decision, df)
  if afficher:
      print(f"AC1={res1} et AC2={res2} et AC3={res3}")
  return res1, res2, res3

# BCi
## BC1 et BC2

In [33]:
def generate_supersets(N, X, V, df):
	candidat = df[df['N_candidat']==N]
	V_sauf_X=[var for var in V if not var in X.keys()]
	supersets = [X]
	for var in V_sauf_X:
		valeur = candidat[var].iloc[0]
		copie = copy.deepcopy(supersets)
		for dictio in copie:
			dictio[var]= valeur
		supersets =supersets + copie
	unique_dicts = {tuple(sorted(d.items())) for d in supersets}
	unique_list_of_dicts = [dict(item) for item in unique_dicts]
	return sorted(unique_list_of_dicts, key=lambda x:len(x))

def BC1(N, X, decision, df=df):
	supersets = generate_supersets(N, X[0], V, df)
	for superset in tqdm(supersets):
		if all(AC(N, superset, decision[0], df)):
			print(f"La actual cause est: {superset} \n")
			return True
	return False

Les performances ne sont pas très bonne, en terme de complexité. Nous proposons une version améliorée.

In [34]:
def BC1_beta(N, X, decision, df=df):
	supersets = generate_supersets(N, X[0], V, df)
	for superset in tqdm(supersets):
		ac1, ac2, ac3 = AC(N, superset, decision[0], df)
		if not ac1:
			return False
		elif not ac2:
			continue
		elif not ac3:
			return False
		return True 
	return False

In [35]:
def test_BC1(f=BC1, only_echec = False):
	display(df)
	# Cas de succès
	if not only_echec:
		assert f("0001", ({"S2": 13}, _), ('A', _))
		assert f("0001", ({"Logique": 1}, _), ('A', _))
		assert f("0005", ({"Logique": 0, "BDD": 0, "PS": 0}, _), ('R', _))

	# Cas d'échec
	assert not f("0001", ({"Logique": 1, "BDD": 1}, _), ('A', _)) #AC3
	assert not f("0004", ({"Logique": 1, "BDD": 1, "S1": 16}, _), ('R', _)) #AC1 n'est pas vérifié
	assert not f("0004", ({"Logique": 0, "BDD": 0, "S1": 16}, _), ('R', _)) # car S1 n'appartent pas à la actual cause
	print("Les tests pour BC1 sont terminés.")

Une autre amélioration: verifier que X peut etre une cause partielle

In [ ]:
def BC1_2_beta(N, X, decision, df=df):
	# verifier que X peut etre une cause partielle
	ac1, ac2, ac3 = AC(N, X, decision, df)
	if not ac1:
		return False
	if ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause
		return False
	if ac2 and ac3: # si X est une actual cause alors c'est une partial cause
		return True
	supersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]
	print("Numéro candidat : ",N)
	for superset in tqdm(supersets):
		#print("\n",superset)
		ac1, ac2, ac3 = AC(N, superset, decision, df)
		if not ac1:
			continue
		if  ac2 and ac3:
			print(f"La actual cause est: {superset} \n")
			return True

	return False

def test_BC1_beta(f=BC1_2_beta, only_echec = True):
	display(df)
	# Cas de succès
	if not only_echec:
		assert f("0001", {"S2": 13}, 'A')
		assert f("0001", {"Logique": 1}, 'A')
		assert f("0005", {"Logique": 0, "BDD": 0, "PS": 0}, 'R')

	# Cas d'échec
	assert not f("0001", {"Logique": 1, "BDD": 1}, 'A') #AC3
	assert not f("0004", {"Logique": 1, "BDD": 1, "S1": 16}, 'R') #AC1 n'est pas vérifié
	assert not f("0004", {"Logique": 0, "BDD": 0, "S1": 16}, 'R') # car S1 n'appartent pas à la actual cause
	print("Les tests pour BC1 sont terminés.")
test_BC1(only_echec=True)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


  0%|          | 0/128 [00:00<?, ?it/s]

Les tests pour BC1 sont terminés.


Nous avons arreté l'éxécution du code de la premiere version

In [36]:
test_BC1()



,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


  0%|          | 0/512 [00:00<?, ?it/s]


La actual cause est: {'S2': 13} 



  0%|          | 0/512 [00:00<?, ?it/s]


La actual cause est: {'Logique': 1} 



  0%|          | 0/128 [00:00<?, ?it/s]

  2%|▏         | 2/128 [00:00<00:03, 34.45it/s]


La actual cause est: {'BDD': 0, 'Logique': 0, 'PS': 0, 'S5': 11} 



 99%|█████████▉| 127/128 [2:05:46<08:41, 521.91s/it]   

In [ ]:
test_BC1(only_echec=False)

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


  0%|          | 0/512 [00:00<?, ?it/s]


La actual cause est: {'S2': 13} 



  0%|          | 0/512 [00:00<?, ?it/s]


La actual cause est: {'Logique': 1} 



  1%|          | 1/128 [00:00<00:14,  8.54it/s]


La actual cause est: {'BDD': 0, 'Logique': 0, 'PS': 0, 'S2': 11} 



  0%|          | 0/128 [00:00<?, ?it/s]

Les tests pour BC1 sont terminés.


## BC3

In [ ]:
def BC3(X, df=None):
    X1, X2 = X

    # Check si les deux X1 et X2 contiennent les memes variables
    if set(X1.keys()) != set(X2.keys()):
        return False

    # Check que l'intersection des deux affectations est vide
    for key in X1.keys():
        if X1[key] == X2[key]:
            return False

    return True

In [ ]:
def test_BC3():
    # Cas de succès
    assert BC3(({"S2":12, "BDD":0}, {"S2":11, "BDD":1}))

    # Cas d'échec
    assert not BC3(({"S1":15, "BDD":1, "Moyenne":2, 'Eligible': False}, {"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))
    assert not BC3(({"S1":15, "BDD":1, 'Eligible': False}, {"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))

    print("BC3 est bien correcte.")

test_BC3()


BC3 est bien correcte.


## BC4

In [7]:
def BC4(N_candidats, X, decisions, df=df):
	def BCi1_3(N_candidats, X, decisions, df=df):
		return BC1_2_beta(N_candidats[0], X[0], decisions[0], df) and BC1_2_beta(N_candidats[1], X[1], decisions[1], df) and BC3(X, df)

	# S'assurer que X verifie tous les Bci1-3
	if not BCi1_3(N_candidats, X, decisions, df):
		return False

	candidat1=df[df.N_candidat==N_candidats[0]].copy()
	candidat2=df[df.N_candidat==N_candidats[1]].copy()
	X1=X[0]
	X2=X[1]
	X_var1=list(X1.keys())
	X_var2=list(X2.keys())

	V_var=list(candidat1.columns)[1:-1] # Prendre toutes les variables qui décrivent un candidat1 sauf N_candidat et Décision

	if X_var1 != X_var2 : # les deux candidats doivent etre testés sur les memes attributs (X)
		return False

	X_var=X_var1
	V_minus_X_var=[var for var in V_var if not var in X_var]


	# Trouver toutes les combinaisons possibles de longueur 1 à la longueur de la liste
	all_combinaisons_var = []

	for nb_var in range(1, len(V_minus_X_var) + 1):
			all_combinaisons_var.extend(combinations(V_minus_X_var, nb_var))

	for var in tqdm(all_combinaisons_var):
		# Remise à l'état initial des sur-ensemble de X1 et X2
		X1_eval=X1.copy()
		X2_eval=X2.copy()
		for v in var:
			X1_eval.update({v: candidat1.iloc[0, candidat1.columns.get_loc(v)]})
			X2_eval.update({v: candidat2.iloc[0, candidat2.columns.get_loc(v)]})
		X_eval=(X1_eval,X2_eval)
		if (BCi1_3(N_candidats, X_eval, decisions, df)):
			return False # Retourner False car on aurait trouvé un superset qui verifie tous les BC1-3



	return True


In [ ]:
def test_BC4():
    # Cas de succès
    display(df)
    assert BC4(("0001","0004"), ({"Logique":"1"}, {"Logique":"0"}), ('A', 'R'))

    print("BC4 est bien correcte.")

test_BC4()

## BC1_2_beta2

In [ ]:
def BC1_2_beta2(N, X, decision, var_notAC_candidat=set(), df=df ):
	# verifier que X peut etre une cause partielle
	ac1, ac2, ac3 = AC(N, X, decision, df)
	if not ac1:
		return False, None
	if ac2 and not ac3: # AC1 et AC2 sont vérifiés et AC3 ne l'est pas --> il existe un sous enesemble de X qui verifie une actual cause donc X ne peut pas etre une partial cause
		return False, None
	if ac2 and ac3: # si X est une actual cause alors c'est une partial cause
		return True, None

  # Créer une structure de données pour enregistrer tous les supersets testés afin d'éviter

	supersets = generate_supersets(N, X, V, df)[1:] #////////////// ajouter [1:]
	print("Numéro candidat : ",N)
	for superset in tqdm(supersets):

		if tuple(superset.keys()) in var_notAC_candidat: # ignorer tous les supersets (passés en entrée) qui ne verifient pas les ACi
			continue

		if not AC1(N, superset,decision):
			return False, var_notAC_candidat
		if  AC2_beta(N, superset,decision):
			print(f"La actual cause est: {superset} \n")
			return True, var_notAC_candidat

		var_notAC_candidat.add(tuple(superset.keys())) # Enregistrer tous les supersets qui ne verifient pas les ACi

	return False, var_notAC_candidat

## BC3_4

In [ ]:
def BC3_4(N_candidats, X, decisions, df=df):
  candidat1=df[df.N_candidat==N_candidats[0]]
  candidat2=df[df.N_candidat==N_candidats[1]]

  # Vérifier que X1 de candidat1 et X2 de candidat2 sont testés sur les memes variables (si ce n'est pas vérifié alors BC3 échoue)
  if X[0].keys()!=X[1].keys():
    return False

  Vmax=[]
  # Générer Vmax : la liste des attributs distincts entre candidat1 et candidat2
  for col in candidat1.columns[1:-1]:
    if candidat1.loc[:,[col]].iloc[0,0]!=candidat2.loc[:,[col]].iloc[0,0]:
      Vmax.append(col)

  # Vérifier si X est inclu dans Vmax (si ce n'est pas vérifié alors BC3 échoue)
  X_var=[ var for var in X[0]]
  if not set(X_var).issubset(set(Vmax)):
    return False

  # Trouver toutes les combinaisons possibles de longueur 1 à la longueur de la liste
  all_combinaisons_var = []

  for nb_var in range(1, len(Vmax) + 1):
      all_combinaisons_var.extend(combinations(Vmax, nb_var))

  all_combinaisons_var_reversed=all_combinaisons_var[::-1]

  var_notAC_candidat1=set()
  var_notAC_candidat2=set()

  # Verifier si cet element est partial cause
  for combinaison in tqdm(all_combinaisons_var_reversed):

    if not set(X_var).issubset(combinaison) : # On en veut évaluer que les supersets de X
      continue

    dict_candidat1=candidat1.loc[:,combinaison].iloc[0].to_dict()
    dict_candidat2=candidat2.loc[:,combinaison].iloc[0].to_dict()

    BC1_2_beta_bool1, var_notAC_candidat1=BC1_2_beta2(N_candidats[0], dict_candidat1, decisions[0],var_notAC_candidat1)
    if not (BC1_2_beta_bool1):
      continue

    BC1_2_beta_bool2, var_notAC_candidat2=BC1_2_beta2(N_candidats[1], dict_candidat2, decisions[1], var_notAC_candidat2)
    if not BC1_2_beta_bool2:
      continue


    # Verifier la BC4
    if len(set(X_var)) < len(combinaison):  # BC4 n'est pas vérifiée car nous avons trouver un superset qui verifie BC1_3
      return False,combinaison

    else : # len(X_var)==len(combinaison)
      return True, X_var

  return False, None # Cas ou c'est un sous ensemble de X_var qui vérifie BC1_4


BC3_4(("0001","0004"), ({"Logique":"1"}, {"Logique":"0"}), ('A', 'R'))

 80%|████████  | 204/255 [00:50<00:00, 166.53it/s]

Numéro candidat :  0004



 98%|█████████▊| 249/255 [00:51<00:01,  4.88it/s] 

AC2 et AC3 est vérifié pour une affectation x_prime: {'BDD': 1, 'Logique': 1}
La actual cause est: {'BDD': 0, 'Logique': 0} 



(True, ['Logique'])

In [ ]:
BC3_4(("0001","0004"), ({"Logique":"1", "BDD":"1"}, {"Logique":"0", "BDD":"0" }), ('A', 'R')) # le subset qui verifie est ({"Logique":"1"}, {"Logique":"0"}))


100%|██████████| 255/255 [00:33<00:00,  7.66it/s] 


(False, None)

# Amélioration spécifique: changement minimal

In [67]:
def AC2_beta_minChanges(N_candidat, X, decision, df=df):
    candidat=df[df.N_candidat==N_candidat].copy()
    if (len(candidat)==0):
        return False
    variable_domains = {
          "S1": range(0, 21),
          "S2": range(0, 21),
          "S3": range(0, 21),
          "S4": range(0, 21),
          "S5": range(0, 21),
          "BDD": [0, 1],
          "Logique": [0, 1],
          "PS": [0, 1],
          "Moyenne":[0,1,2],
          "Eligible":[True,False]
      }

    if(decision == 'A'):
      variable_domains = {
          "S1": range(0, candidat.loc[:,['S1']].iloc[0,0])[::-1],
          "S2": range(0,  candidat.loc[:,['S2']].iloc[0,0])[::-1],
          "S3": range(0,  candidat.loc[:,['S3']].iloc[0,0])[::-1],
          "S4": range(0,  candidat.loc[:,['S4']].iloc[0,0])[::-1],
          "S5": range(0,  candidat.loc[:,['S5']].iloc[0,0])[::-1],
          "BDD": [0, 1],
          "Logique": [0, 1],
          "PS": [0, 1],
          "Moyenne":[0,1,2],
          "Eligible":[True,False]
      }
    elif(decision == 'R'):
      variable_domains = {
          "S1": range(candidat.loc[:,['S1']].iloc[0,0]+1, 21),
          "S2": range(candidat.loc[:,['S2']].iloc[0,0]+1, 21),
          "S3": range(candidat.loc[:,['S3']].iloc[0,0]+1,21),
          "S4": range(candidat.loc[:,['S4']].iloc[0,0]+1,21),
          "S5": range(candidat.loc[:,['S5']].iloc[0,0]+1,21),
          "BDD": [0, 1],
          "Logique": [0, 1],
          "PS": [0, 1],
          "Moyenne":[0,1,2],
          "Eligible":[True,False]
      }
    elif(decision == 'L'):
      variable_domains = {
          "S1": list(range(candidat.loc[:,['S1']].iloc[0,0]+1, 21))+list(range(0, candidat.loc[:,['S1']].iloc[0,0])[::-1]),
          "S2": list(range(candidat.loc[:,['S2']].iloc[0,0]+1, 21))+list(range(0, candidat.loc[:,['S2']].iloc[0,0])[::-1]),
          "S3": list(range(candidat.loc[:,['S3']].iloc[0,0]+1, 21))+list(range(0, candidat.loc[:,['S3']].iloc[0,0])[::-1]),
          "S4": list(range(candidat.loc[:,['S4']].iloc[0,0]+1, 21))+list(range(0, candidat.loc[:,['S4']].iloc[0,0])[::-1]),
          "S5": list(range(candidat.loc[:,['S5']].iloc[0,0]+1, 21))+list(range(0, candidat.loc[:,['S5']].iloc[0,0])[::-1]),
          "BDD": [0, 1],
          "Logique": [0, 1],
          "PS": [0, 1],
          "Moyenne":[0,1,2],
          "Eligible":[True,False]
      }

    def subdict_combinations(dictionary):
        if not dictionary:
            return [{}]

        key = next(iter(dictionary))
        rest = dictionary.copy()
        del rest[key]

        subcombos = subdict_combinations(rest)
        result = []
        for subcombo in subcombos:
            result.append(subcombo)
            with_key = {key: dictionary[key]}
            result.append({**with_key, **subcombo})
        return result

    def generate_combinations(X, variable_domains):
        # Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
        combinations = product(*[variable_domains[var] for var in X])

        # Exclure la combinaison identique à X
        return [combination for combination in combinations if combination != tuple(X.values())]
    sub=subdict_combinations(X)
    for subX in sub:
        if len(subX):
            X_var=list(subX.keys())
            vals=candidat.iloc[0].to_dict()
            combinations = generate_combinations(subX, variable_domains)
            # Vérifier chaque combinaison
            for combination in combinations:
                x_prime = dict(zip(X_var, combination))
                for k in x_prime.keys():
                    vals[k]=x_prime[k]
                m,e,d=med(**{k: v for k, v in vals.items() if k in ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']})

                if ('Eligible' in x_prime and x_prime['Eligible']!=e) or ('Moyenne' in x_prime and x_prime['Moyenne']!=m) :
                    continue
                if d!=decision and len(x_prime)==len(X):
                    print("AC2 et AC3 est vérifié pour une affectation x_prime:",x_prime, " la decision est :", d)
                    return True
                elif d!=decision:
                    print("Erreur : AC2 et AC3 est vérifié pour une affectation x_prime :", x_prime, " la decision est :", d)
                    return False
    print("Erreur : AC2 n'est pas vérifiée, veuillez saisir un superset de l'ensemble des variables sélectionné")
    return False
#ca ameliore les cas où AC3 n'est pas vérifié tout en ayant une meme complexité dans le cas où AC2 et AC3 sont verifiés

In [71]:
def test_AC2_beta_minChanges():
	# Cas de succès
	assert AC2_beta_minChanges("0001", {"S1": 15},'A')
	assert AC2_beta_minChanges("0002", {"S1": 11, "S5": 10}, 'R')

	# Cas d'échec
	assert not AC2_beta_minChanges("0004", {"S5": 15, "BDD": 0},'R')
	assert not AC2_beta_minChanges("0005", {"S1": 11, "BDD": 0}, 'R')
	assert not AC2_beta_minChanges("0003", {"Logique": 1, "S5": 13, "S2":12},'L')

	print("AC2 est bien correcte")

test_AC2_beta_minChanges()

AC2 et AC3 est vérifié pour une affectation x_prime: {'S1': 11}  la decision est : L
AC2 et AC3 est vérifié pour une affectation x_prime: {'S1': 15, 'S5': 20}  la decision est : L
Erreur : AC2 n'est pas vérifiée, veuillez saisir un superset de l'ensemble des variables sélectionné
Erreur : AC2 n'est pas vérifiée, veuillez saisir un superset de l'ensemble des variables sélectionné
Erreur : AC2 et AC3 est vérifié pour une affectation x_prime : {'Logique': 0}  la decision est : R
AC2 est bien correcte
